In [ ]:
%pip install matplotlib pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 375.0 kB/s  0:00:23m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 478.5 kB/s  0:00:27m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 322.2 kB/s  0:00:14 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 469.6 kB/s  0:00:03 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 429.9 kB/s  0:00:44m0:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 328.0 kB/s  0:00:21 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [matplotlib]1 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the columns as they appear in your CSV files
cols = ['Rows1', 'Cols1', 'Rows2', 'Cols2', 'Time']

# Load GPU data
try:
    df_gpu = pd.read_csv('cudaTime.csv', names=cols, header=None)
    df_cpu = pd.read_csv('cpuTime.csv', names=cols, header=None)
except FileNotFoundError:
    print("Error: Make sure cudaTime.csv and cpuTime.csv are in the same folder.")

# Merge using all four dimension columns
# We suffix the time columns to distinguish them
df = pd.merge(
    df_gpu, 
    df_cpu, 
    on=['Rows1', 'Cols1', 'Rows2', 'Cols2'], 
    suffixes=('_GPU', '_CPU')
)

# Create a readable label for the x-axis (e.g., "512x512")
df['SizeLabel'] = df['Rows1'].astype(str) + "x" + df['Cols2'].astype(str)

print("Data successfully merged. First 5 rows:")
print(df[['SizeLabel', 'Time_GPU', 'Time_CPU']].head())

Error: Make sure cudaTime.csv and cpuTime.csv are in the same folder.


NameError: name 'df_gpu' is not defined

In [ ]:
plt.figure(figsize=(12, 6))

# Plotting the lines
plt.plot(df['SizeLabel'], df['Time_GPU'], marker='o', color='cyan', label='GPU (CUDA)', linewidth=2)
plt.plot(df['SizeLabel'], df['Time_CPU'], marker='s', color='magenta', label='CPU (Sequential)', linewidth=2)

# Graph styling
plt.yscale('log') # Logarithmic scale is essential for performance gaps
plt.title('Execution Time: GPU vs. CPU (Log Scale)', fontsize=14)
plt.xlabel('Matrix Dimensions', fontsize=12)
plt.ylabel('Execution Time (ms)', fontsize=12)
plt.grid(True, which="both", ls="--", alpha=0.5)
plt.legend()

plt.show()

In [ ]:
# Calculate speedup
df['Speedup'] = df['Time_CPU'] / df['Time_GPU']

plt.figure(figsize=(12, 5))
bars = plt.bar(df['SizeLabel'], df['Speedup'], color='teal', alpha=0.8)

# Adding labels on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.1, f'{yval:.1f}x', ha='center', va='bottom', fontsize=10)

plt.title('GPU Speedup Factor over CPU', fontsize=14)
plt.xlabel('Matrix Dimensions', fontsize=12)
plt.ylabel('Speedup (Multiple of CPU Speed)', fontsize=12)
plt.grid(axis='y', linestyle=':', alpha=0.7)

plt.show()